# 测试Tester环境下的简单合约

In [1]:
import json
from web3 import Web3
from solcx import compile_source

## 定义合约demo

In [2]:
compiled_sol = compile_source(
     '''
     pragma solidity >0.5.0;

     contract Greeter {
         string public greeting;

         constructor() public {
             greeting = 'Hello';
         }

         function setGreeting(string memory _greeting) public {
             greeting = _greeting;
         }

         function greet() view public returns (string memory) {
             return greeting;
         }
     }
     ''',
     output_values=['abi', 'bin']
 )

contract_id, contract_interface = compiled_sol.popitem()

bytecode = contract_interface['bin']
abi = contract_interface['abi']
open(f'res_dir/meta.txt', 'w').write(json.dumps(abi))

549

In [3]:
# web3.py instance
w3 = Web3(Web3.EthereumTesterProvider())
w3

In [4]:
w3.isConnected()

True

In [5]:
w3.eth.accounts

['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']

In [6]:
for acc in w3.eth.accounts:
    print(w3.fromWei(w3.eth.get_balance(acc), 'ether'))

1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000


In [7]:
w3.eth.get_block('latest')

AttributeDict({'number': 0,
 'hash': HexBytes('0x9da270c828731ccafde10d88c9925e1b11416777e5c1c1bb8406e07361cd0a90'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x000000000000002a'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logs_bloom': 0,
 'transactionsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'receipts_root': '0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421',
 'stateRoot': HexBytes('0xf1588db9a9f1ed91effabdec31f93cb4212b008c8b8ba047fd55fabebf6fd727'),
 'miner': '0x0000000000000000000000000000000000000000',
 'difficulty': 131072,
 'totalDifficulty': 131072,
 'size': 517,
 'extraData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 30029122,
 'gasUsed': 0,
 'timestamp': 1655971920,
 'transactions': [],
 'uncles': [],
 'baseFeePerGas': 1000000000})

In [8]:
# set pre-funded account as sender
for i in range(0, 4):
    w3.eth.default_account = w3.eth.accounts[i]
    f = open(f'accounts/{w3.eth.default_account}.txt', 'w')

    f.write(f'{w3.eth.default_account}\n\n')

    Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)

    # Submit the transaction that deploys the contract
    tx_hash = Greeter.constructor().transact()

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    greeter = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
    f.write(greeter.functions.greet().call())

    tx_hash = greeter.functions.setGreeting('Nihao').transact()
    f.write(f'{tx_hash}\n\n')

    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    f.write(f'{tx_receipt}\n\n')

    f.write(f'{greeter.functions.greet().call()}\n\n')
    
    f.close()